In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [131]:
### Input Parameters
def create_placeholders():
    x = tf.placeholder(dtype=tf.float32, shape=[None,784])
    y_true = tf.placeholder(dtype=tf.float32, shape=[None,10])
    return x, y_true

  
  
### Helper Function
def initialize_weights(shape):
  init_random_dist = tf.truncated_normal(shape, stddev=0.1, initializer=tf.contrib.layers.xavier_initializer())
  return tf.Variable(init_random_dist)

def initialize_bias(shape):
  init_bias_vals = tf.constant(0.1, shape=shape)
  return tf.Variable(init_bias_vals)

def convolution_2d(x, W, stride=1):
  return tf.nn.conv2d(input=x, filter=W, strides=[1, stride, stride, 1], padding='SAME', name=None)

def convolution_layer(x, shape, activation='relu'):
  W = initialize_weights(shape)
  b = initialize_bias(len(shape))
  if activation=='leaky_relu':
    return tf.nn.leaky_relu(convolution_2d(x, W) + b)
  return tf.nn.relu(convolution_2d(x, W) + b)

#avg pooling or max pooling with 2 strides
def pooling_2d(x, pooling_type='max_pooling', size=2, stride=2):
  if pooling_type == 'avg_pooling':
    return tf.nn.avg_pool(x, ksize=[1,size,size,1], strides=[1, stride, stride,1], padding='SAME', name=None)
  return tf.nn.max_pool(x, ksize=[1,size,size,1], strides=[1, stride, stride,1], padding='SAME', name=None)

def cost(Z3, Y):
  return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z3, labels=Y), name=None)

def reshape_image(x, size=28):
  return tf.reshape(x,[-1, size, size, 1])

'\ndef forward_propagation: return None\ndef training_network: return None\n'

In [0]:
#Hyperparameters
hyperparameters = {'learning_rate': 0.0001, 
                   'num_epochs': 5000, 
                   'print_cost': True}

In [0]:
#Training the Network
def training_network(mnist, hyperparameters):
  
  learning_rate = hyperparameters['learning_rate']
  num_epoch = hyperparameters['num_epochs']
  print_cost = hyperparameters['print_cost']
  
  x, y = create_placeholder()
  x_image = reshape_image(x)
  
  conv_1 = convolutional_layer(x_image,shape=[6,6,1,32])
  conv_1_pooling = pooling_2d(conv_1)
  
  conv_2 = convolutional_layer(conv_1_pooling,shape=[6,6,32,64])
  conv_2_pooling = pooling_2d(conv_2)
  
  conv_2_flat = tf.reshape(conv_2_pooling,[-1,7*7*64])
  full_layer_one = tf.nn.relu(normal_full_layer(conv_2_flat,1024))
  
  hold_prob = tf.placeholder(tf.float32)
  full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)
  
  y_pred = normal_full_layer(full_one_dropout,10)
  
  cost_fn = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,
                                                                    logits=y_pred))
  
  train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost_fn)
  
  init = tf.global_variables_initializer()

  with tf.Session() as sess:
      sess.run(init)
      for i in range(num_epoch):
          batch_x , batch_y = mnist.train.next_batch(50)
          sess.run(train,feed_dict={x:batch_x,
                                    y_true:batch_y, 
                                    hold_prob:0.5})
          
          if print_cost == True and i%50 == 0:
              print('Currently on step {}'.format(i))
              print('Accuracy is:')

              matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
              acc = tf.reduce_mean(tf.cast(matches,tf.float32))

              print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels, hold_prob:1.0}))
              print('\n')
              
      plt.plot(np.squeeze(costs))
      plt.ylabel('cost')
      plt.xlabel('iterations (per tens)')
      plt.title("Learning rate =" + str(learning_rate))
      plt.show() 

In [0]:
training_network(mnist)